# 1. Importamos librerias

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

# 2. Importamos dataSet

In [4]:
df = pd.read_csv("train.csv")
ds_trabajo = df.copy()
ds_trabajo.head(5)

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo


# 3. Dividir conjunto train y test

In [5]:
x_train, x_test, y_train, y_test = train_test_split(ds_trabajo.review_es, ds_trabajo.sentimiento, test_size = 0.30, random_state = 25, shuffle = True)

# 4. Instancio, entreno y testeo modelos

Se entrenaran modelos Naive Bayes Multinomial y previamente se utilizaran los metodos Bag of Words e Inverse document frequency para evaluar que procesamiento de los datos conviene mas. A priori tendria que ser mas conveniente usar IDF.

In [6]:
model_tfid = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_CV = make_pipeline(CountVectorizer(), MultinomialNB())

model_tfid.fit(x_train, y_train)
model_CV.fit(x_train, y_train)

model_tfid_prediction = model_tfid.predict(x_test)
model_CV_prediction = model_CV.predict(x_test)

# 5. Evaluo y visualizo resultados

In [7]:
print("Matrix de confusion de TFID:")
print(confusion_matrix(y_test, model_tfid_prediction))

print("Matrix de confusion de CV:")
print(confusion_matrix(y_test, model_CV_prediction))

print()

print("TFID Metrics:")
print(metrics.classification_report(y_test, model_tfid_prediction))

print("CV Metrics:")
print(metrics.classification_report(y_test, model_CV_prediction))


Matrix de confusion de TFID:
[[6517 1082]
 [1116 6285]]
Matrix de confusion de CV:
[[6664  935]
 [1501 5900]]

TFID Metrics:
              precision    recall  f1-score   support

    negativo       0.85      0.86      0.86      7599
    positivo       0.85      0.85      0.85      7401

    accuracy                           0.85     15000
   macro avg       0.85      0.85      0.85     15000
weighted avg       0.85      0.85      0.85     15000

CV Metrics:
              precision    recall  f1-score   support

    negativo       0.82      0.88      0.85      7599
    positivo       0.86      0.80      0.83      7401

    accuracy                           0.84     15000
   macro avg       0.84      0.84      0.84     15000
weighted avg       0.84      0.84      0.84     15000



Tal y como preveimos, TFI es superior.

# 5.1 Busqueda de los mejores hiperparametros usando TfidfVectorizer

In [8]:
parameters = {
    'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'multinomialnb__alpha': [0.1, 0.5, 1.0],
}

optimized = make_pipeline(TfidfVectorizer(), MultinomialNB())

grid_search = GridSearchCV(optimized, parameters, cv=5, n_jobs=-1)

grid_search.fit(x_train, y_train)

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)

# Evaluar el rendimiento accuracy en el conjunto de prueba
accuracy = grid_search.score(x_test, y_test)
print(f"Precisión en el conjunto de prueba: {accuracy}")

best_model = grid_search.best_estimator_

Mejores parámetros encontrados:
{'multinomialnb__alpha': 0.5, 'tfidfvectorizer__ngram_range': (2, 2)}
Precisión en el conjunto de prueba: 0.8770666666666667


# 6. Pasamos a evaluar con el conjunto test real

In [9]:
dft = pd.read_csv("test.csv")
ds_test = dft.copy()

ds_test_ID = ds_test.ID
ds_test_for_predict = ds_test.review_es

test_prediction = best_model.predict(ds_test_for_predict)

ds_submission = pd.DataFrame({'ID': ds_test['ID'], 'sentimiento': test_prediction})
ds_submission.to_csv('./prediccion_final.csv', index=False)